In [ ]:
import logging
import sys

logger = logging.getLogger()
logger.setLevel(logging.INFO)

if (not logger.hasHandlers()):

    ch = logging.StreamHandler(sys.stdout)
    ch.setLevel(logging.INFO)
    logger.addHandler(ch)

In [ ]:
from pathlib import Path

year: int = 2019
tid: str = 'h09v05'
startDay: int = 1
endDay: int = 366
sensors = set(['MOD'])

# ---
# Paths
# ---
burnDir = Path('/css/modis/Collection6.1/L3/MCD64A1-BurnArea/')
modDir = Path('/css/modis/Collection6.1/L2G')
outDir = Path('/explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old')
postProcDir = Path('/explore/nobackup/projects/ilab/data/MODIS/ancillary/MODIS_postprocess_mask/MODIS_postprocess_mask_v70.0.2')

# ---
# Use this to switch between the original and new subcondition1 equations.
# ---
useNewSubcondition1: bool = False

# ---
# Set to True to run the post-processing steps.
# ---
runPostProcessing: bool = True

# ---
# This is for finding imports.  Point this one level above your local
# repository.
# ---
sys.path.append('/explore/nobackup/people/rlgill/innovation-lab-repositories/')

In [9]:
import numpy as np

from osgeo import gdal
from osgeo.osr import SpatialReference

from modis_water.model.BandReader import BandReader as br
from modis_water.model.Classifier import Classifier
from modis_water.model.MaskGenerator import MaskGenerator


# -----------------------------------------------------------------------------
# class DebugSimpleClassifier
# -----------------------------------------------------------------------------
class DebugSimpleClassifier(Classifier):

    CLASSIFIER_NAME = 'DebugSimple'

    # -------------------------------------------------------------------------
    # __init__
    #
    # Debug is true, so the masks are written and available to be applied to
    # the equation images.
    # -------------------------------------------------------------------------
    def __init__(self, year, tile, outDir, modDir, startDay=1, endDay=365,
                 logger=None, sensors=set([br.MOD]), debug=True):

        inBands = [br.SOLZ, br.STATE, br.SR1, br.SR2, br.SR3, br.SR4, br.SR5, 
                   br.SR6, br.SR7]
        
        super(DebugSimpleClassifier, self). \
            __init__(year, 
                     tile, 
                     str(outDir), 
                     str(modDir), 
                     startDay, 
                     endDay, 
                     logger,
                     debug=debug,
                     sensors=sensors,
                     inBands=inBands)  #,
                     # dataType=np.uint8,
                     # badData=250)

    # -------------------------------------------------------------------------
    # getClassifierName
    # -------------------------------------------------------------------------
    def getClassifierName(self):
        return DebugSimpleClassifier.CLASSIFIER_NAME

    # -------------------------------------------------------------------------
    # _runOneSensorOneDay
    # -------------------------------------------------------------------------
    def _runOneSensorOneDay(self, bandDict, outName):

        # Mark requested this currently unused variable to be defined.
        red = bandDict[br.SR1]
        
        # Name the arrays as named in water_change.c
        nir = bandDict[br.SR2]
        blue = bandDict[br.SR3]
        swir5 = bandDict[br.SR5]
        swir7 = bandDict[br.SR7]

        # ---
        # Add NDVI to bandDict.
        # NDVI is normally calculated with range -1,1. This multiplies
        # that range by 10,000 making it an integer-friendly range.
        # The NDVI conditions listed in water_change.c are multiplied
        # by 10,000 to match the range of the computed NDVI.
        # ---
        ndvi = self.computeNdvi(bandDict[br.SR1], bandDict[br.SR2])
        ndviBadCalculation = (bandDict[br.SR1] + bandDict[br.SR2]) == 0

        # ---
        # Define the rules as masks.
        # ---
        if useNewSubcondition1:
            
            subcondition1 = (swir5 >= 453) & (blue > 675) & (nir > 1000)
        
        else:
            subcondition1 = (swir5 >= 453) & (blue < 675) & (nir > 1000)

        land1 = (swir5 < 1017) & (swir7 < 773) & subcondition1
        water1 = (swir5 < 1017) & (swir7 < 773) & ~subcondition1

        water2 = (swir5 >= 1017) & (nir < 1777) & (ndvi < 825) & \
                 (blue < 651)

        land2 = (swir5 >= 1017) & (nir < 1777) & (ndvi < 825) & \
            (blue >= 651)

        land3 = (swir5 >= 1017) & (nir < 1777) & (ndvi >= 825) & \
                (ndvi < 4125) & (nir >= 1329) & (swir7 < 1950)

        land4 = (swir5 >= 1017) & (nir < 1777) & (ndvi >= 825) & \
                (ndvi >= 4125)

        land5 = (swir5 >= 1017) & (nir >= 1777)
        land6 = (swir5 < 1017) & (swir7 >= 773)

        waterConditions = water1 | water2
        landConditions = land1 | land2 | land3 | land4 | land5 | land6

        # Apply the model.
        predictions = np.full((br.COLS, br.ROWS), self._noData)      
        predictions[waterConditions] = Classifier.WATER  # 1
        predictions[landConditions] = Classifier.LAND    # 0

        predictions = np.where(ndviBadCalculation,
                               self._noData,
                               predictions)
        
        # Encode and write classification equations.
        newName = Path(outName).with_suffix('.equations.tif')

        self.writeClassificationEquations(land1, land2, land3, land4, land5, 
                                          land6, water1, water2, newName)

        return predictions
    
    # -------------------------------------------------------------------------
    # writeClassificationEquations
    # -------------------------------------------------------------------------
    def writeClassificationEquations(self, land1, land2, land3, land4,
                                     land5, land6, water1, water2,
                                     outName) -> None:
        
        equations = np.full((br.COLS, br.ROWS), self._noData)
        equations[land1] = 1
        equations[land2] = 2
        equations[land3] = 3
        equations[land4] = 4
        equations[land5] = 5
        equations[land6] = 6
        equations[water1] = 7
        equations[water2] = 8
        
        # Retrieve the masks written due to debug = True.
        genlMaskPath = Path(self._outDir) / 'GeneralMask.tif'
        landMaskPath = Path(self._outDir) / 'LandMask.tif'
        
        generalMask = gdal.Open(str(genlMaskPath)).ReadAsArray()
        landMask = gdal.Open(str(landMaskPath)).ReadAsArray()
        
        # Mimic Classifier.
        generalMaskedImage = \
            np.where(generalMask == MaskGenerator.GOOD_DATA,
                     equations,
                     Classifier.BAD_DATA).astype(self._npDt)
                                      
        predictedLandAndMasked = ((generalMaskedImage == Classifier.LAND) &
                                  (landMask == MaskGenerator.BAD_DATA))

        finalImage = np.where(predictedLandAndMasked,
                              self._badData,
                              generalMaskedImage).astype(self._npDt)

        self._createOutputImage(str(outName), finalImage)   

In [10]:
classifier = DebugSimpleClassifier(year,
                                   tid,
                                   outDir,
                                   modDir,
                                   startDay=startDay, 
                                   endDay=endDay,
                                   logger=logger)

classifier.run()
logger.info('Classifier completed')

Reading MOD tile h09v05 for day 1
Creating /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/2019-001-h09v05-MOD-DebugSimple.tif
Generating mask
SR1 < -100: True
SR2 < -100: True
Aero: True
Cloudy: True
Cloud mixed: True
Cloud shadow: True
Cloud internal: True
Mask type: int16
Classifiying
Writing image as 3
GDT_Int16 is 3
SR5 type:int16
Predictions type: int64
Masking
Writing image as 3
GDT_Int16 is 3
Final image type: int16
Reading MOD tile h09v05 for day 2
Creating /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/2019-002-h09v05-MOD-DebugSimple.tif
Generating mask
SR1 < -100: True
SR2 < -100: True
Aero: True
Cloudy: True
Cloud mixed: True
Cloud shadow: True
Cloud internal: True
Mask type: int16
Classifiying
Writing image as 3
GDT_Int16 is 3
SR5 type:int16
Predictions type: int64
Masking
Writing image as 3
GDT_Int16 is 3
Final image type: int16
Reading MOD tile h09v05 for day 3
Creating /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v

/explore/nobackup/people/rlgill/innovation-lab-repositories/modis_water/model/Classifier.py:131: RuntimeWarning: invalid value encountered in divide
  (((sr2 - sr1) / (sr2 + sr1)) * 10000).astype(self._npDt)


Writing image as 3
GDT_Int16 is 3
SR5 type:int16
Predictions type: int64
Masking
Writing image as 3
GDT_Int16 is 3
Final image type: int16
Reading MOD tile h09v05 for day 117
Creating /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/2019-117-h09v05-MOD-DebugSimple.tif
Generating mask
SR1 < -100: True
SR2 < -100: True
Aero: True
Cloudy: True
Cloud mixed: True
Cloud shadow: True
Cloud internal: True
Mask type: int16
Classifiying
Writing image as 3
GDT_Int16 is 3
SR5 type:int16
Predictions type: int64
Masking
Writing image as 3
GDT_Int16 is 3
Final image type: int16
Reading MOD tile h09v05 for day 118
Creating /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/2019-118-h09v05-MOD-DebugSimple.tif
Generating mask
SR1 < -100: True
SR2 < -100: True
Aero: True
Cloudy: True
Cloud mixed: True
Cloud shadow: True
Cloud internal: True
Mask type: int16
Classifiying
Writing image as 3
GDT_Int16 is 3
SR5 type:int16
Predictions type: int64
Masking
Writing image as 3


In [11]:
if runPostProcessing:

    from modis_water.model.AnnualMap import AnnualMap
    from modis_water.model.BurnScarMap import BurnScarMap
    from modis_water.model.QAMap import QAMap
    from modis_water.model.SevenClass import SevenClassMap

    logger.info('Creating annual map.')
    
    for sensor in sensors:
        
        annualMapPath = AnnualMap.createAnnualMap(outDir,
                                                  year,
                                                  tid,
                                                  sensor,
                                                  classifier.getClassifierName(),
                                                  logger,
                                                  georeferenced=True)
        
        logger.info('Annual map: ' + str(annualMapPath))

        # Burn scar map
        logger.info('Creating annual burn scar map.')
        
        postAnnualBurnScarPath = BurnScarMap.generateAnnualBurnScarMap(
            sensor,
            year,
            tid,
            burnDir,
            classifier.getClassifierName(),
            outDir,
            logger)
                    
        logger.info('Burn scar map: ' + str(postAnnualBurnScarPath))

        # QA map
        logger.info('Post processing.')
        
        postAnnualPath = QAMap.generateQA(sensor,
                                          year,
                                          tid,
                                          postAnnualBurnScarPath,
                                          str(postProcDir),
                                          annualMapPath,
                                          classifier.getClassifierName(),
                                          outDir,
                                          logger,
                                          geoTiff=True,
                                          georeferenced=True)

        logger.info('QA map: ' + str(postAnnualPath))
                    
        # Seven class
        SevenClassMap.generateSevenClass(sensor,
                                         year,
                                         tid,
                                         str(postProcDir),
                                         postAnnualPath,
                                         classifier.getClassifierName(),
                                         outDir,
                                         logger,
                                         geoTiff=True,
                                         georeferenced=True)
        
classifier._logger.info('Complete')

Creating annual map.
Day image does not exist: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/2019-366-h09v05-MOD-DebugSimple.tif


/explore/nobackup/people/rlgill/innovation-lab-repositories/modis_water/model/AnnualMap.py:74: RuntimeWarning: invalid value encountered in divide
  (sumWater / (sumWater + sumLand) * 100).astype(np.int16),


Annual map: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/2019-h09v05-MOD-DebugSimple-Mask.tif
Creating annual burn scar map.
Wrote annual burn scar map to: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/MOD.A2019.h09v05.DebugSimple.AnnualBurnScar.20250241044.tif
Burn scar map: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/MOD.A2019.h09v05.DebugSimple.AnnualBurnScar.20250241044.tif
Post processing.


/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


Wrote annual QA products to: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/MOD44W.A2019.h09v05.DebugSimple.AnnualWaterProduct.20250241044.tif
Wrote annual QA products to: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/MOD44W.A2019.h09v05.DebugSimple.AnnualWaterProductQA.20250241044.tif
QA map: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/MOD44W.A2019.h09v05.DebugSimple.AnnualWaterProduct.20250241044.tif
Wrote annual seven class to: /explore/nobackup/projects/ilab/test/subcondition1-tests/h09v05-old/MOD44W.A2019.h09v05.DebugSimple.AnnualSevenClass.20250241044.tif
Complete
